<a href="https://colab.research.google.com/github/bigluck07/Code_States_Project/blob/main/n213a_ridge_regression%5B%EC%9D%B4%EB%8C%80%EC%9A%B4%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *DATA SCIENCE / SECTION 2 / SPRINT 1 / NOTE 3*

# 📝 Assignment
---

# Ridge Regression

이번 모듈에서 배운 내용을 조금 다른 데이터를 통해 복습해 봅시다. 데이터에 대한 정보는 다음 링크를 참조해 주세요.

- [Melbourne Housing Market]('https://www.kaggle.com/anthonypino/melbourne-housing-market')

이번에도 Price 변수를 target으로 정하고 모델을 구성해 봅시다.

In [ ]:
import sys
!{sys.executable} -m pip install -U pandas-profiling[notebook]
!jupyter nbextension enable --py widgetsnbextension

In [ ]:
!pip install pandas_profiling --upgrade

In [ ]:
pip install --upgrade scikit-learn

In [ ]:
pip install category_encoders

In [74]:
!pip list | grep scikit-learn

scikit-learn                  0.24.1        


In [6]:
import pandas as pd
from pandas_profiling import ProfileReport # 변수들에 대한 통계량을 한눈에 보기 쉽게 리포팅 해줌
from sklearn.model_selection import train_test_split

# get data
df = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/melbourne_house_prices/MELBOURNE_HOUSE_PRICES_LESS.csv')

# drop high cardinality cols
df.drop(columns=['Suburb','Address','SellerG','Date'], inplace=True)

# drop na values in Price
df.dropna(subset=['Price'], inplace=True)

# drop duplicated rows
df.drop_duplicates(inplace=True)

In [7]:
df.head()

,Rooms,Type,Price,Method,Postcode,Regionname,Propertycount,Distance,CouncilArea
0,3,h,1490000.0,S,3067,Northern Metropolitan,4019,3.0,Yarra City Council
1,3,h,1220000.0,S,3067,Northern Metropolitan,4019,3.0,Yarra City Council
2,3,h,1420000.0,S,3067,Northern Metropolitan,4019,3.0,Yarra City Council
3,3,h,1515000.0,S,3040,Western Metropolitan,1543,7.5,Moonee Valley City Council
4,2,h,670000.0,S,3042,Western Metropolitan,3464,10.4,Moonee Valley City Council


### 1) 데이터의 특성을 파악하기 위해 pandas_profiling을 사용해 보세요.
- **범주형 특성으로 원핫인코딩이 꼭 필요한 특성의 갯수를 과제 제출폼에 제출하세요.**

In [ ]:
profile = ProfileReport(df, minimal=True)
profile

카테고리 데이터 4개

### 2) 범주형 변수들에 대해 one-hot encoding(category_encoders)을 수행하고 데이터를 train/test 셋으로 무작위로 나누어 주세요.(train/test = 80%/20%)
- [category_encoders.one_hot.OneHotEncoder](https://contrib.scikit-learn.org/category_encoders/onehot.html#one-hot), 파라미터는 use_cat_names = True 사용
- 데이터 분리는 [sklearn.model_selection.train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html?highlight=train%20test%20split#sklearn-model-selection-train-test-split) 을 사용해 주세요. 사용해야 하는 파라미터는 test_size, train_size, random_state=2 입니다. 나머지는 default 사용합니다.
- **분리된 학습데이터의 첫번째 샘플의 인덱스 값를 과제 제출폼에 제출하세요.**


In [62]:
from category_encoders import OneHotEncoder

## 원핫 인코딩
encoder = OneHotEncoder(use_cat_names = True)
# 범주형 데이터의 인코딩 여기서는 type, methodm regionname, councilarea
df_OneHot = encoder.fit_transform(df) 
df_OneHot.shape

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


(42515, 55)

In [63]:
# 특성 간 독립성을 위한 불필요 피쳐 제거
df_OneHot = feature.drop(['Type_t', 'Method_SA','Regionname_Western Victoria','CouncilArea_Murrindindi Shire Council'],axis=1)
df_OneHot.shape

(42515, 51)

In [66]:
# train, test 데이터 분리 8:2

train, test = train_test_split(df_OneHot, test_size = 0.2, random_state=2)

train.shape , test.shape

((34012, 51), (8503, 51))

In [67]:
# target 값 제외 - 트레이
X_train = train.drop(columns = 'Price')
y_train = train['Price']
X_test = test.drop(columns = 'Price')
y_test = test['Price']
train.shape , test.shape

((34012, 51), (8503, 51))

In [68]:
train.head() # 첫번째 인덱스 = 602

,Rooms,Price,Type_h,Type_u,Method_S,Method_SP,Method_VB,Method_PI,Postcode,Regionname_Northern Metropolitan,Regionname_Western Metropolitan,Regionname_Southern Metropolitan,Regionname_South-Eastern Metropolitan,Regionname_Eastern Metropolitan,Regionname_Northern Victoria,Regionname_Eastern Victoria,Propertycount,Distance,CouncilArea_Yarra City Council,CouncilArea_Moonee Valley City Council,CouncilArea_Port Phillip City Council,CouncilArea_Brimbank City Council,CouncilArea_Darebin City Council,CouncilArea_Hobsons Bay City Council,CouncilArea_Stonnington City Council,CouncilArea_Boroondara City Council,CouncilArea_Monash City Council,CouncilArea_Kingston City Council,CouncilArea_Maroondah City Council,CouncilArea_Bayside City Council,CouncilArea_Banyule City Council,CouncilArea_Glen Eira City Council,CouncilArea_Whitehorse City Council,CouncilArea_Maribyrnong City Council,CouncilArea_Hume City Council,CouncilArea_Moreland City Council,CouncilArea_Manningham City Council,CouncilArea_Melbourne City Council,CouncilArea_Melton City Council,CouncilArea_Frankston City Council,CouncilArea_Casey City Council,CouncilArea_Greater Dandenong City Council,CouncilArea_Nillumbik Shire Council,CouncilArea_Whittlesea City Council,CouncilArea_Knox City Council,CouncilArea_Wyndham City Council,CouncilArea_Yarra Ranges Shire Council,CouncilArea_Cardinia Shire Council,CouncilArea_Macedon Ranges Shire Council,CouncilArea_Mitchell Shire Council,CouncilArea_Moorabool Shire Council
602,4,1750000.0,1,0,1,0,0,0,3106,0,0,0,0,1,0,0,6202,15.5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
40851,5,900000.0,1,0,1,0,0,0,3169,0,0,0,1,0,0,0,2727,17.5,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
57233,4,530000.0,1,0,0,0,1,0,3028,0,1,0,0,0,0,0,7630,15.5,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8816,4,975000.0,1,0,1,0,0,0,3033,0,1,0,0,0,0,0,5629,11.7,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3481,3,556000.0,1,0,1,0,0,0,3076,1,0,0,0,0,0,0,10926,19.6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [69]:
test.head()

,Rooms,Price,Type_h,Type_u,Method_S,Method_SP,Method_VB,Method_PI,Postcode,Regionname_Northern Metropolitan,Regionname_Western Metropolitan,Regionname_Southern Metropolitan,Regionname_South-Eastern Metropolitan,Regionname_Eastern Metropolitan,Regionname_Northern Victoria,Regionname_Eastern Victoria,Propertycount,Distance,CouncilArea_Yarra City Council,CouncilArea_Moonee Valley City Council,CouncilArea_Port Phillip City Council,CouncilArea_Brimbank City Council,CouncilArea_Darebin City Council,CouncilArea_Hobsons Bay City Council,CouncilArea_Stonnington City Council,CouncilArea_Boroondara City Council,CouncilArea_Monash City Council,CouncilArea_Kingston City Council,CouncilArea_Maroondah City Council,CouncilArea_Bayside City Council,CouncilArea_Banyule City Council,CouncilArea_Glen Eira City Council,CouncilArea_Whitehorse City Council,CouncilArea_Maribyrnong City Council,CouncilArea_Hume City Council,CouncilArea_Moreland City Council,CouncilArea_Manningham City Council,CouncilArea_Melbourne City Council,CouncilArea_Melton City Council,CouncilArea_Frankston City Council,CouncilArea_Casey City Council,CouncilArea_Greater Dandenong City Council,CouncilArea_Nillumbik Shire Council,CouncilArea_Whittlesea City Council,CouncilArea_Knox City Council,CouncilArea_Wyndham City Council,CouncilArea_Yarra Ranges Shire Council,CouncilArea_Cardinia Shire Council,CouncilArea_Macedon Ranges Shire Council,CouncilArea_Mitchell Shire Council,CouncilArea_Moorabool Shire Council
60324,4,1062000.0,1,0,1,0,0,0,3172,0,0,0,1,0,0,0,3940,22.2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20098,3,1306000.0,1,0,1,0,0,0,3130,0,0,0,0,1,0,0,5713,13.4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14980,3,811000.0,1,0,0,0,0,0,3137,0,0,0,0,1,0,0,4654,26.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
32414,3,1721000.0,0,0,1,0,0,0,3141,0,0,1,0,0,0,0,14887,2.7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
54957,4,820000.0,1,0,0,0,1,0,3056,1,0,0,0,0,0,0,11918,5.2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### 3) SelectKBest를 사용해서 20개의 특성만 selection 하세요
- score_func 에는 f_regression을 사용합니다 # api 확인후 하기
- **가장 스코어(Attributes: scores_ 사용)가 높게 나온 특성의 이름을 과제 제출폼에 제출하세요.**

In [70]:
# target(Price)와 가장 correlated 된 features 를 k개 고르는 것이 목표입니다.
## f_regresison, SelectKBest
from sklearn.feature_selection import f_regression, SelectKBest

## selctor 정의합니다.
selector = SelectKBest(score_func=f_regression, k=20)

## 학습데이터에 fit_transform 
X_train_selected = selector.fit_transform(X_train, y_train)

## 테스트 데이터는 transform
X_test_selected = selector.transform(X_test)


X_train_selected.shape, X_test_selected.shape

((34012, 20), (8503, 20))

In [71]:
all_names = X_train.columns

## selector.get_support()
selected_mask = selector.get_support()

## 선택된 특성들
selected_names = all_names[selected_mask]
print('Selected names: ', selected_names)


Selected names:  Index(['Rooms', 'Type_h', 'Type_u', 'Method_SP', 'Method_VB',
       'Regionname_Northern Metropolitan', 'Regionname_Western Metropolitan',
       'Regionname_Southern Metropolitan',
       'Regionname_South-Eastern Metropolitan', 'Distance',
       'CouncilArea_Brimbank City Council',
       'CouncilArea_Stonnington City Council',
       'CouncilArea_Boroondara City Council',
       'CouncilArea_Bayside City Council', 'CouncilArea_Hume City Council',
       'CouncilArea_Manningham City Council',
       'CouncilArea_Melton City Council', 'CouncilArea_Frankston City Council',
       'CouncilArea_Whittlesea City Council',
       'CouncilArea_Wyndham City Council'],
      dtype='object')


In [ ]:
import numpy as np
array = selector.scores_

pd.DataFrame(dict(feature_names=X_train.columns,
                  scores = selector.scores_))\
                  .sort_values('scores',ascending = False)
# 스코어가 가장 높은 피쳐 = Rooms

### 4) RidgeCV를 사용해서 회귀모델을 만들어 보세요
- attribute는 normalize=True, cv=5를 사용하세요. 나머지는 default
- **alphas = [0, 0.001, 0.01, 0.1, 1] 을 시도해보고 가장 좋은 점수가 나온 alpha 값을 과제 제출폼에 제출하세요.**

In [77]:
from sklearn.linear_model import RidgeCV

alphas = [0, 0.001, 0.01, 0.1, 1]

ridge = RidgeCV(alphas=alphas, normalize=True, cv=5)
ridge.fit(X_train_selected, y_train)
print("alpha: ", ridge.alpha_)
print("best score: ", ridge.best_score_)
# 가장좋은 점수의 alpha = 0.001

alpha:  0.001
best score:  0.5510507608242426


### 5) 테스트셋에 대한 MAE 값을 계산하고 모델이 잘 만들어졌는지 확인해 보세요.
- **테스트셋에 대한 MAE 값을 과제 제출폼에 제출하세요.**

In [81]:
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.linear_model import Ridge

for alpha in [0, 0.001, 0.01, 0.1, 1]:
    print('\n')
    print(f'Ridge Regression, alpha={alpha}')

    # Ridge 모델 학습
    model = Ridge(alpha=alpha, normalize=True)  
    model.fit(X_train_selected, y_train)
    y_pred = model.predict(X_test_selected)

    # MAE for test
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    print(f'Test MAE: ${mae:,.0f}')
    print(f'Test R2: {r2:,.5f}')
# alpha 0.001의 MAE = $262,857



Ridge Regression, alpha=0
Test MAE: $262,917
Test R2: 0.56916


Ridge Regression, alpha=0.001
Test MAE: $262,857
Test R2: 0.56914


Ridge Regression, alpha=0.01
Test MAE: $262,375
Test R2: 0.56884


Ridge Regression, alpha=0.1
Test MAE: $260,253
Test R2: 0.56217


Ridge Regression, alpha=1
Test MAE: $286,398
Test R2: 0.45906


## 🔥 도전과제

### 6) best ridge regression model을 만드세요.
    - 특성의 갯수에 제한이 없습니다.
    - 다항회귀를 사용하세요
    - 여러 람다값을 시도하세요
    - 람다 0인 경우를 비교하세요
    - RidgeCV를 사용하세요
    - 만든 모델의 특징을 간략히 설명하고 테스트세트 성능 결과를 메신저에서 공유하세요

In [ ]:
### 이곳에서 과제를 진행해 주세요 ###